<center><h1 style="color:blue; font-style: italic;"> Schnorr Protocol</h1></center>

In [12]:
import hashlib

### 1. parametres et generation des cles   

In [24]:
def find_generator(p, q):
    g_full = primitive_root(p)
    g = power_mod(g_full, (p - 1)//q, p)
    if Mod(g, p).multiplicative_order() == q:
        return g
    else:
        return None  # ou relancer
        
def generate_parameters(q_bits=160, p_bits=1024):
    # q premier
    q = random_prime(2^q_bits, lbound=2^(q_bits - 1))

    # Trouver p = kq + 1 premier (q|p-1)
    while True:
        k = randint(2^(p_bits - q_bits - 1), 2^(p_bits - q_bits))
        p = k * q + 1
        if is_prime(p):
            break

    g = find_generator(p,q)

    return p, q, g

def keygen(p, q, g):
    x = randint(1, q - 1)  #cle privée
    y = power_mod(g, x, p) #cle publique
    return x, y



### 2. Signature

In [25]:
def H(m, t, q):
    """
    Fonction de hachage : SHA-256(t || m) mod q
    """
    m_bytes = m.encode()
    t_bytes = Integer(t).digits(base=256)
    h = hashlib.sha256(bytes(t_bytes[::-1]) + m_bytes).hexdigest()
    return int(h, 16) % q

def sign(m, x, p, q, g):
    """
    Signature de Schnorr : retourne (e, s)
    """
    r = randint(1, q - 1)
    t = power_mod(g, r, p)
    e = H(m, t, q)
    s = (r + e * x) % q
    return e, s

### 3. Verification

In [26]:
def verify(m, e, s, y, p, q, g):
    """
    Vérifie une signature Schnorr
    """
    t_prime = (power_mod(g, s, p) * power_mod(y, -e, p)) % p
    e_prime = H(m, t_prime, q)
    return e == e_prime

## Test

In [ ]:
# Générer les paramètres publics
p, q, g = generate_parameters()

# Générer une paire de clés
x, y = keygen(p, q, g)

# Signer un message
message = "bonjour"
e, s = sign(message, x, p, q, g)

# Vérifier la signature
valid = verify(message, e, s, y, p, q, g)

# Affichage
print("Message signé :", message)
print("Signature : (e =", e, ", s =", s, ")")
print("Signature valide ?", valid)
